In [1]:
from sentence_transformers import SentenceTransformer,InputExample,losses,evaluation,util
import pandas as pd
import numpy as np
import torch
import argparse
import pandas as pd
import numpy as np
from torch.utils.data import DataLoader
from sklearn import preprocessing
from sklearn.utils.class_weight import compute_class_weight

In [2]:
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
# print all output to a file
import sys
import os
import datetime
now = datetime.datetime.now()

data = 'R8_toy'
random_pairs = 20
positive_pairs = 5
EPS = 1e-9
model_name = 'all-MiniLM-L6-v2'
# model_name = 'all-mpnet-base-v2'
data_dir = 'data/'+data+'/'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using device:', device)
sys.stdout = open('output_'+ data+'_baseline_'+model_name+now.strftime("%Y-%m-%d-%H-%M-%S")+'.txt', 'w')
sbert_model = SentenceTransformer(model_name)

Using device: cpu


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
state_dict = torch.load("/content/sbert_for_task_1_30 (1).pth", map_location=torch.device('cpu'))

<ipython-input-4-f199cfb0e644>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/content/sbert_for_task_1_30 (1).pth", map_location=torch.device('cp

In [5]:
sbert_model.load_state_dict(state_dict)

<All keys matched successfully>

In [6]:
train = pd.read_csv(r'/content/train.csv')
train = train.sample(frac=1).reset_index(drop=True)
label_encoder = preprocessing.LabelEncoder()
X_train = train['text'].values
y_train = train['label'].values
y_train = label_encoder.fit_transform(y_train)

In [7]:
val = pd.read_csv(r'/content/val.csv')
X_val = val['text'].values
y_val = val['label'].values
y_val = label_encoder.fit_transform(y_val)
#print(X_train.shape, y_train.shape, X_val.shape, y_val.shape)
num_labels = len(np.unique(y_train))
#print(num_labels)

In [8]:
test = pd.read_csv(r'/content/test.csv')
X_test = test['text'].values
y_test = test['label'].values
y_test = label_encoder.fit_transform(y_test)

In [9]:
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
y_val = torch.tensor(y_val, dtype=torch.long).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)

In [10]:
X_train = X_train.tolist()
X_val = X_val.tolist()
X_test = X_test.tolist()
total_X = X_train + X_val+ X_test
total_X = np.array(total_X)


In [11]:
features = sbert_model.encode(total_X)

In [12]:
features = torch.tensor(features)

In [13]:
import scipy.sparse as sp
import torch.nn as nn
import math
def normalize(adj):
    """Symmetrically normalize adjacency matrix."""
    adj = sp.coo_matrix(adj)
    rowsum = np.array(adj.sum(1))
    d_inv_sqrt = np.power(rowsum, -0.5).flatten()
    d_inv_sqrt[np.isinf(d_inv_sqrt)] = 0.
    d_mat_inv_sqrt = sp.diags(d_inv_sqrt)
    return adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo()

In [14]:
def sparse_mx_to_torch_sparse_tensor(sparse_mx):
  sparse_mx = sparse_mx.tocoo().astype(np.float32)
  indices = torch.from_numpy(np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
  values = torch.from_numpy(sparse_mx.data)
  shape = torch.Size(sparse_mx.shape)
  return torch.sparse.FloatTensor(indices, values, shape)

In [15]:
adj = torch.mm(features, features.T)
adj = normalize(adj)
adj = sparse_mx_to_torch_sparse_tensor(adj)
adj = adj.to(device)

<ipython-input-14-d42694959cde>:6: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:651.)
  return torch.sparse.FloatTensor(indices, values, shape)


In [16]:
features = features.to(device)

In [17]:
class GraphConvolution(nn.Module):

    def __init__(self, in_features, out_features, bias=True):
        super(GraphConvolution, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.FloatTensor(in_features, out_features))
        if bias:
            self.bias = nn.Parameter(torch.FloatTensor(out_features))
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()

    def reset_parameters(self):
        stdv = 1. / math.sqrt(self.weight.size(1))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.uniform_(-stdv, stdv)

    def forward(self, input, adj):
        support = torch.mm(input, self.weight)
        output = torch.spmm(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'

In [18]:
SEED = 123
torch.manual_seed(SEED)
np.random.seed(SEED)
import torch.nn.functional as F
import torch.optim as optim

test_layer = GraphConvolution(5, 4)
print("Weight Matrix", test_layer.weight)
print("Bias", test_layer.bias )
print(test_layer.in_features,test_layer.out_features)

In [19]:
class GCN(nn.Module):
    def __init__(self, nfeat, nhid, nclass, dropout):
        super(GCN, self).__init__()

        self.gc1 = GraphConvolution(nfeat, nhid)
        self.gc2 = GraphConvolution(nhid, nclass)
        self.dropout = dropout

    def forward(self, x, adj):
        x = F.relu(self.gc1(x, adj))
        x = F.dropout(x, self.dropout, training=self.training)
        x = self.gc2(x, adj)
        return F.log_softmax(x, dim=1)

In [20]:
def intialize_model(nfeat, hidden, c, dropout, lr, wd):
  # Model and optimizer
  model = GCN(nfeat=features.shape[1],
              nhid=hidden,
              nclass=c,
              dropout=dropout)
  optimizer = optim.Adam(model.parameters(),
                        lr=lr, weight_decay=wd)


  # # Print model's state_dict
  # print("Model's state_dict:")
  # for param_tensor in model.state_dict():
  #     print(param_tensor, "\t", model.state_dict()[param_tensor].size())

  # # Print optimizer's state_dict
  # print("Optimizer's state_dict:")
  # for var_name in optimizer.state_dict():
  #     print(var_name, "\t", optimizer.state_dict()[var_name])

  return model, optimizer


In [21]:
def accuracy(output, labels):
  preds   = output.max(1)[1].type_as(labels)
  print(preds)
  print(labels)
  correct = preds.eq(labels).double()
  correct = correct.sum()
  return correct / len(labels)

In [22]:
idx_train = [i for i in range(len(X_train))]
idx_val = [i for i in range(len(X_train),len(X_train)+len(X_val))]
idx_test =[i for i in range(len(X_train)+len(X_val), len(X_train)+len(X_val)+len(X_test))]

In [23]:
l , w, h, d = 0.01,0.005,16,0.5
model, optimizer = intialize_model(features.shape[1], h, 8, d, l, w)
state_dict = torch.load("/content/gcn_for_task_1.pth", map_location=torch.device('cpu'))
model.load_state_dict(state_dict)

<ipython-input-23-b8f5bae7f32b>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("/content/gcn_for_task_1.pth", map_location=torch.device('cpu'))


<All keys matched successfully>

In [24]:
model.eval()
output = model(features, adj)
loss_test = F.nll_loss(output[idx_test], y_test)
acc_test = accuracy(output[idx_test], y_test)
print("Test set results:",
      "loss= {:.4f}".format(loss_test.item()),
      "accuracy= {:.4f}".format(acc_test.item()))

In [25]:
acc_test

tensor(0.3194, dtype=torch.float64)